## This notebook is about predicting students' performance in their final exam based on first period exam G1 and second period exam G2

In [1]:
import sklearn
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
df = pd.read_csv('Datasets/Regression/student-por.csv', usecols = [30,31,32])

In [5]:
df.head()

,G1,G2,G3
0,0,11,11
1,9,11,11
2,12,13,12
3,14,14,14
4,11,13,13


In [27]:
def getHypothesis(X, theta):
    return np.dot(X, theta)

In [7]:
def getCost(X, Y, theta):
    H = getHypothesis(X, theta)
    cost = sum(pow(H-Y,2))/(2*len(Y))
    return cost

In [12]:
X = df[['G1','G2']]
Y = df[['G3']]

In [13]:
X.head()

,G1,G2
0,0,11
1,9,11
2,12,13
3,14,14
4,11,13


### Inserting first column in the training set as ones for ease in the upcoming operations like getting hypothesis and working with gradient descent

In [14]:
X.insert(0,'ones',1)

In [15]:
X.head()

,ones,G1,G2
0,1,0,11
1,1,9,11
2,1,12,13
3,1,14,14
4,1,11,13


### converting training data from data frame into numpy arrays

In [16]:
train_matrix = X.as_matrix()

In [17]:
type(train_matrix)

numpy.ndarray

In [18]:
train_matrix

array([[ 1,  0, 11],
       [ 1,  9, 11],
       [ 1, 12, 13],
       ..., 
       [ 1, 11, 12],
       [ 1, 10, 10],
       [ 1, 10, 11]], dtype=int64)

In [20]:
ground_truth_matrix = Y.values
type(ground_truth_matrix)

numpy.ndarray

#### Testing the hypothesis function

In [28]:
getHypothesis(train_matrix, [1,1,1])

array([12, 21, 26, 29, 25, 25, 26, 24, 32, 25, 29, 23, 26, 25, 29, 35, 27,
       28, 17, 25, 26, 24, 26, 21, 22, 22, 24, 23, 25, 24, 22, 31, 28, 26,
       25, 23, 29, 27, 24, 28, 23, 22, 30, 20, 22, 22, 26, 35, 24, 26, 28,
       31, 20, 26, 26, 26, 30, 31, 28, 32, 34, 21, 27, 28, 26, 32, 24, 20,
       22, 31, 25, 21, 25, 27, 23, 23, 24, 27, 19, 24, 23, 20, 24, 26, 26,
       24, 25, 31, 23, 19, 21, 28, 26, 28, 24, 27, 21, 26, 26, 26, 19, 33,
       26, 22, 33, 21, 21, 28, 23, 31, 28, 23, 19, 36, 20, 31, 32, 29, 24,
       28, 29, 28, 27, 26, 24, 19, 22, 20, 29, 26, 22, 20, 22, 24, 26, 29,
       19, 25, 24, 28, 20, 28, 23, 24, 24, 19, 21, 28, 18, 21, 18, 29, 24,
       27, 22, 17, 25, 22, 23, 26, 23, 19, 24, 21, 20, 27, 21, 27, 16, 18,
       22, 23, 21, 18, 17, 14, 18, 18, 17, 16, 20, 35, 25, 33, 19, 34, 21,
       28, 29, 22, 26, 21, 20, 23, 22, 24, 36, 28, 30, 25, 20, 28, 26, 19,
       25, 24, 22, 22, 26, 30, 27, 22, 30, 22, 29, 24, 30, 27, 27, 18, 23,
       30, 26, 24, 25, 26